<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#压缩分组键，group_keys=False" data-toc-modified-id="压缩分组键，group_keys=False-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>压缩分组键，group_keys=False</a></span></li><li><span><a href="#分位数与桶分析" data-toc-modified-id="分位数与桶分析-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>分位数与桶分析</a></span></li><li><span><a href="#示例：使用指定分组值填充缺失值" data-toc-modified-id="示例：使用指定分组值填充缺失值-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>示例：使用指定分组值填充缺失值</a></span></li><li><span><a href="#示例：分组加权平均和相关性" data-toc-modified-id="示例：分组加权平均和相关性-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>示例：分组加权平均和相关性</a></span></li><li><span><a href="#示例：逐组线性回归" data-toc-modified-id="示例：逐组线性回归-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>示例：逐组线性回归</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
tips = pd.read_csv('examples/tips.csv')

In [3]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [4]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [5]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [6]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:] # 最后n行

In [7]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [8]:
# 以'smoker'为分组键，然后调用top函数
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [9]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [10]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [11]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [12]:
tips.groupby('smoker')['tip_pct'].size()

smoker
No     151
Yes     93
Name: tip_pct, dtype: int64

In [13]:
result.unstack('smoker') # 行→列

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

# 压缩分组键，group_keys=False

In [14]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


# 分位数与桶分析

In [15]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})

In [16]:
quartiles = pd.cut(frame.data1, 4) # 将 data1 分成4组

In [17]:
quartiles

0       (0.0553, 1.809]
1       (0.0553, 1.809]
2       (-3.46, -1.699]
3       (0.0553, 1.809]
4       (0.0553, 1.809]
5       (0.0553, 1.809]
6       (0.0553, 1.809]
7      (-1.699, 0.0553]
8       (0.0553, 1.809]
9      (-1.699, 0.0553]
10     (-1.699, 0.0553]
11      (-3.46, -1.699]
12      (-3.46, -1.699]
13     (-1.699, 0.0553]
14     (-1.699, 0.0553]
15      (0.0553, 1.809]
16      (0.0553, 1.809]
17      (0.0553, 1.809]
18     (-1.699, 0.0553]
19     (-1.699, 0.0553]
20      (0.0553, 1.809]
21     (-1.699, 0.0553]
22      (0.0553, 1.809]
23     (-1.699, 0.0553]
24     (-1.699, 0.0553]
25     (-1.699, 0.0553]
26     (-1.699, 0.0553]
27     (-1.699, 0.0553]
28     (-1.699, 0.0553]
29      (0.0553, 1.809]
             ...       
970    (-1.699, 0.0553]
971     (0.0553, 1.809]
972    (-1.699, 0.0553]
973    (-1.699, 0.0553]
974    (-1.699, 0.0553]
975      (1.809, 3.563]
976    (-1.699, 0.0553]
977     (0.0553, 1.809]
978     (-3.46, -1.699]
979    (-1.699, 0.0553]
980     (0.0553,

In [18]:
def get_stats(group):
    return {'min': group.min(),
            'max': group.max(),
            'count': group.count(),
            'mean': group.mean()}

In [19]:
grouped = frame.data2.groupby(quartiles)

In [20]:
grouped.apply(get_stats).unstack() # 行→列

,count,max,mean,min
data1,,,,
"(-3.46, -1.699]",33.0,1.460690,-0.112112,-2.417108
"(-1.699, 0.0553]",466.0,2.685644,0.008918,-2.778695
"(0.0553, 1.809]",472.0,2.897478,-0.049558,-4.510443
"(1.809, 3.563]",29.0,1.366897,-0.265375,-1.737247


In [21]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [22]:
grouped = frame.data2.groupby(grouping)

In [23]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,1.955039,-0.046792,-2.417108
1,100.0,2.590508,0.067193,-2.518434
2,100.0,2.581511,0.017681,-2.665943
3,100.0,2.053946,-0.089653,-2.778695
4,100.0,2.685644,0.067884,-2.228054
5,100.0,2.271970,0.052074,-2.867096
6,100.0,2.897478,-0.032425,-2.401952
7,100.0,2.894595,-0.089707,-2.568936
8,100.0,2.005168,-0.125655,-2.294637


# 示例：使用指定分组值填充缺失值

In [24]:
s = Series(np.random.randn(6))

In [25]:
s

0    1.309085
1    0.680646
2   -0.835270
3    0.572450
4   -0.538425
5    0.779690
dtype: float64

In [26]:
s[::2] = np.nan

In [27]:
s

0         NaN
1    0.680646
2         NaN
3    0.572450
4         NaN
5    0.779690
dtype: float64

In [28]:
# 用平均值来填充缺失值
s.fillna(s.mean())

0    0.677595
1    0.680646
2    0.677595
3    0.572450
4    0.677595
5    0.779690
dtype: float64

In [29]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregen', 'Nevada', 'California', 'Idaho']

In [30]:
group_key = ['East'] * 4 + ['West'] * 4

In [31]:
data = Series(np.random.randn(8),
              index=states)

In [32]:
data

Ohio         -0.207426
New York     -0.256013
Vermont      -1.515922
Florida       1.671018
Oregen        1.277467
Nevada        0.678857
California    1.244771
Idaho        -0.496293
dtype: float64

In [33]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [34]:
data

Ohio         -0.207426
New York     -0.256013
Vermont            NaN
Florida       1.671018
Oregen        1.277467
Nevada             NaN
California    1.244771
Idaho              NaN
dtype: float64

In [35]:
data.groupby(group_key).mean()

East    0.402526
West    1.261119
dtype: float64

In [36]:
fill_mean = lambda g: g.fillna(g.mean)

In [37]:
data.groupby(group_key).apply(fill_mean)

Ohio                                                  -0.207426
New York                                              -0.256013
Vermont       <bound method Series.mean of Ohio       -0.207...
Florida                                                 1.67102
Oregen                                                  1.27747
Nevada        <bound method Series.mean of Oregen        1.2...
California                                              1.24477
Idaho         <bound method Series.mean of Oregen        1.2...
dtype: object

In [38]:
fill_values = {'East': 0.5, 'West': -1}

In [39]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [40]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.207426
New York     -0.256013
Vermont       0.500000
Florida       1.671018
Oregen        1.277467
Nevada       -1.000000
California    1.244771
Idaho        -1.000000
dtype: float64

# 示例：分组加权平均和相关性

In [41]:
df = DataFrame({'category': list('aaaabbbb'),
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})

In [42]:
df

,category,data,weights
0,a,0.620360,0.254960
1,a,0.084968,0.231628
2,a,-0.032146,0.743794
3,a,0.296534,0.354218
4,b,-1.530620,0.316555
5,b,-1.659705,0.743773
6,b,0.124325,0.416396
7,b,-1.614980,0.035830


In [43]:
grouped = df.groupby('category')

In [44]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [45]:
grouped.apply(get_wavg)

category
a    0.163433
b   -1.140499
dtype: float64

In [46]:
grouped.agg(get_wavg)

,data,weights
category,,
a,0.163433,0.163433
b,-1.140499,-1.140499


In [47]:
close_px = pd.read_csv('examples/stock_px_2.csv',
                       parse_dates=True, # 解析index时间
                       index_col=0) # 第一列为index

In [48]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [49]:
close_px.describe()

,AAPL,MSFT,XOM,SPX
count,2214.000000,2214.000000,2214.000000,2214.000000
mean,125.516147,23.945452,59.558744,1183.773311
std,107.394693,3.255198,16.725025,180.983466
min,6.560000,14.330000,26.210000,676.530000
25%,37.135000,21.700000,49.492500,1077.060000
50%,91.455000,24.000000,62.970000,1189.260000
75%,185.605000,26.280000,72.510000,1306.057500
max,422.000000,34.070000,87.480000,1565.150000


In [50]:
close_px[-4:] # 最后4行

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [51]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [52]:
# pct_change() 用于计算变化率
rets = close_px.pct_change().dropna()

In [53]:
get_year = lambda x: x.year

In [54]:
by_year = rets.groupby(get_year)

In [55]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [56]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# 示例：逐组线性回归

In [57]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [58]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514
